In [ ]:
# DREAMTEAM PROJECT 1

In [ ]:
# # Datasets to be used
# 1. Trading cards (high liquidity cards) - 
# 2. Bitcoin - https://api.alternative.me/v2/ticker/Bitcoin/?convert=CAD
# 3. Government bond (baseline) - https://au.investing.com/rates-bonds/australia-10-year-bond-yield-historical-data
# 4. ASX200 (general stock market) - https://www.google.com/finance/quote/XJO:INDEXASX
# 5. Property - ABS data in 8 capital cities- https://www.abs.gov.au/statistics/economy/price-indexes-and-inflation/residential-property-price-indexes-eight-capital-cities/latest-release

In [10]:
# Intialising imports
import os
import requests
import pandas as pd
import json
import datetime as dt
from pathlib import Path

%matplotlib inline

In [11]:
# Importing Bitcoin current price
# Bitcoin API URL
btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=AUD"

# Fetch current BTC price
btc_response = requests.get(btc_url)
btc_content = btc_response.content
btc_data = btc_response.json()
#print (json.dumps(btc_data, indent=4))
btc_price_current = float (btc_data["data"]["1"]["quotes"]["USD"]["price"])
print (btc_price_current)

39853.0


In [29]:
# Importing Bitcoin historical data

# Set the file path
btc_path=Path("btc_historical.csv")

# Reading the data
btc_df=pd.read_csv(btc_path, index_col='Date', parse_dates=True, infer_datetime_format=True)
btc_df.index = btc_df.index.date

# Ensuring data loaded correctly
btc_df.head()

,Close
2015-11-19,454.097675
2015-11-20,445.560711
2015-11-22,447.867676
2015-11-23,449.365998
2015-11-24,442.422744


In [26]:
# Cleaning Bitcoin historical data

# Count nulls
btc_df.isnull().mean()

# Drop nulls
btc_df = btc_df.dropna()

# Checking nulls are dropped
btc_df.isnull().sum()

Close    0
dtype: int64

In [31]:
# Importing and cleaning ASX200 historical data

# Set the file path
asx200_path=Path("asx200_historical.csv")

# Reading the data
asx200_df=pd.read_csv(btc_path, index_col='Date', parse_dates=True, infer_datetime_format=True)
asx200_df.index = asx200_df.index.date

# Ensuring data loaded correctly
asx200_df.head()

,Close
2015-11-19,454.097675
2015-11-20,445.560711
2015-11-22,447.867676
2015-11-23,449.365998
2015-11-24,442.422744


In [32]:
asx200_df.isnull().mean()

Close    0.0
dtype: float64

In [34]:
# Importing and cleaning bonds historical data

# Set the file path
bonds_path=Path("asx200_historical.csv")

# Reading the data
bonds_df=pd.read_csv(btc_path, index_col='Date', parse_dates=True, infer_datetime_format=True)
bonds_df.index = bonds_df.index.date

# Ensuring data loaded correctly
bonds_df.head()

,Close
2015-11-19,454.097675
2015-11-20,445.560711
2015-11-22,447.867676
2015-11-23,449.365998
2015-11-24,442.422744


In [35]:
bonds_df.isnull().mean()

Close    0.0
dtype: float64

In [36]:
# Importing and cleaning property historical data

# Set the file path
property_path=Path("asx200_historical.csv")

# Reading the data
property_df=pd.read_csv(btc_path, index_col='Date', parse_dates=True, infer_datetime_format=True)
property_df.index = property_df.index.date

# Ensuring data loaded correctly
property_df.head()

,Close
2015-11-19,454.097675
2015-11-20,445.560711
2015-11-22,447.867676
2015-11-23,449.365998
2015-11-24,442.422744


In [37]:
property_df.isnull().mean()

Close    0.0
dtype: float64

In [39]:
# All Nulls removed from data sets

print (f"BTC is {btc_df.isnull().sum()}")
print (f"asx200 is {asx200_df.isnull().sum()}")
print (f"bonds is {bonds_df.isnull().sum()}")
print (f"property is {property_df.isnull().sum()}")

BTC is Close    0
dtype: int64
asx200 is Close    0
dtype: int64
bonds is Close    0
dtype: int64
property is Close    0
dtype: int64
